In [7]:
pip list

Package                 Version
----------------------- -----------
absl-py                 2.1.0
aiohappyeyeballs        2.4.3
aiohttp                 3.10.5
aiosignal               1.2.0
appnope                 0.1.4
asttokens               3.0.0
astunparse              1.6.3
attrs                   24.2.0
blinker                 1.6.2
Brotli                  1.0.9
cachetools              5.3.3
certifi                 2024.8.30
cffi                    1.17.1
charset-normalizer      3.3.2
click                   8.1.7
comm                    0.2.2
contourpy               1.3.1
cryptography            41.0.3
cycler                  0.12.1
debugpy                 1.8.10
decorator               5.1.1
exceptiongroup          1.2.2
executing               2.1.0
flatbuffers             24.3.25
fonttools               4.55.3
frozenlist              1.5.0
gast                    0.4.0
google-auth             2.29.0
google-auth-oauthlib    0.5.2
google-pasta            0.2.0
grpcio            

In [3]:
pip install pandas==1.5.2

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 1.9 MB/s eta 0:00:00a 0:00:01
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pickle
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, RNN, GRUCell, Dropout
import sklearn
import pandas as pd

In [5]:

def load_rnn():
    with open('images_hz/model_RNN.pkl','rb') as f:  # Python 3: open(..., 'rb')
        _, model, _, _ = pickle.load(f)
    return model


def load_svc():
    with open('images_hz/model_SVC.pkl','rb') as f:
        model, _, _ = pickle.load(f)
    return model

def loadModels(classifier):
    if classifier == 'RNN':
        clf = load_rnn()
    elif classifier == 'SVC':
        clf = load_svc()
    return clf

def scores(clf, user_input_word):
    return clf.predict(user_input_word)


In [ ]:
import re
import unicodedata
import nltk
from cleantext import clean
from nltk.corpus import stopwords
from nltk.stem.snowball import EnglishStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('punkt_tab')

In [ ]:
stop_words = {'a','about','above','after','again','against','ain','all','am','an','and','any','are','aren',"aren't",'as','at',
'be','because','been','before','being','below','between','both','but','by','can','couldn',"couldn't",'d','did','didn',"didn't",
'do','does','doesn',"doesn't",'doing','don',"don't",'down','during','each','few','for','from','further','had','hadn',"hadn't",
'has','hasn',"hasn't",'have','haven',"haven't",'having','he','her','here','hers','herself','him','himself','his','how','i','if',
'in','into','is','isn',"isn't",'it',"it's",'its','itself','just','ll','m','ma','me','mightn',"mightn't",'more','most','mustn',"mustn't",
'my','myself','needn',"needn't",'no','nor','not','now','o','of','off','on','once','only','or','other','our','ours','ourselves','out',
'over','own','re','s','same','shan',"shan't",'she',"she's",'should',"should've",'shouldn',"shouldn't",'so','some','such','t','than',
'that',"that'll",'the','their','theirs','them','themselves','then','there','these','they','this','those','through','to','too',
'under','until','up','ve','very','was','wasn',"wasn't",'we','were','weren',"weren't",'what','when','where','which','while','who',
'whom','why','will','with','won',"won't",'wouldn',"wouldn't",'y','you',"you'd","you'll","you're","you've",'your','yours','yourself',
'yourselves'}
stop_words.update(['the', 'and','for', 'from', 'was', 'what', 'with', 'this', \
                'that',  'don', 'pure', 'lot', 'are', 'who', 'more', 'will', 'tab', \
                'each' , 'would', 'but', 'not','its','all','your', 'last','over', \
                    'are','you', 'can', 'above', 'his','she','ready', 'yes', \
                    'size'])
stop_words.remove("no")
stemmer = EnglishStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatization(words) :
    output = []
    for string in words :
        lemma = wordnet_lemmatizer.lemmatize(string)
        if (lemma not in output) : output.append(lemma)
    return output

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def clean_text(text):
    text = clean(
        text = text,
        fix_unicode = True,
        to_ascii=True,
        lower = True,
        replace_with_url = ' ',
        replace_with_email = ' ',
        lang = 'en'
    )
    # Remove all special characters and punctuation
    text = re.sub(r"[^A-Za-z0-9\s]+", " ", text)
    # Remove repeated characters
    text = re.sub(r'(.)\1{3,}',r'\1', text)
    # remove extra spaces, tabs, and new lines
    text = " ".join(text.split())
    w = str(text)
    w = unicode_to_ascii(w.lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z0-9?.!]+", " ", w)
    w = re.sub(r'\b\w{0,2}\b', '', w)
    # stem the word
    words = word_tokenize(w.strip())
    words2 = [stemmer.stem(word) for word in words]
    # lemmatize the word
    words3 = lemmatization(words2)

    # remove stopword
    words = [word for word in words3 if word not in stop_words]

    if len(words) < 1: # sometimes, all words are removed
        return w
    else:
        return ' '.join(words).strip()

In [ ]:


# use the st.selectbox() method to choose between the RandomForest classifier, the SVM classifier and the LogisticRegression classifier. Then return to the Streamlit web application to view the select box.
option = 'SVC'
clf = loadModels(option)
user_input_word = 'Merry Christmas!'
output_st = scores(clf, user_input_word)

/Users/hui/anaconda3/envs/tf/lib/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/hui/anaconda3/envs/tf/lib/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator GridSearchCV from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


ValueError: could not convert string to float: 'Merry Christmas!'